In [ ]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.optimizers import SGD, Adam

In [ ]:
class_names = ['monkey','person']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
nb_classes = len(class_names)
IMAGE_SIZE = (256, 256)

In [ ]:
def load_data():
    datasets = ['archive/training_set', 'archive/test_set']#資料夾
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                if file == "desktop.ini":
                    break
                # Open and resize the img
                print(img_path)
                image = cv2.imread(img_path)
                print(image.shape)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #cv讀照片，顏色莫認為BGR，需轉為RGB，錯誤表示黑白或已轉
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
print(train_images)

In [ ]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

In [ ]:
train_images = train_images / 255.0 
test_images = test_images / 255.0

In [ ]:
input_shape = (256, 256, 3)

In [ ]:
model = Sequential([
    Conv2D(256, (3, 3), input_shape=input_shape, padding='same',
           activation='relu', strides=2),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Dropout(0.2),
    Conv2D(512, (3, 3), input_shape=input_shape, padding='same',
           activation='relu', strides=2),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Dropout(0.2),
    Flatten(),
    Dropout(0.5),
    Dense(2, activation='softmax') #輸出層，分類用softmax
])
model.compile(optimizer = 'adam', #SGD(lr=0.1)
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, 
                    #validation_data=(test_images, test_labels),
                    #verbose=2,callbacks=[earlyStop],
                    batch_size=8, epochs=100)

In [ ]:
plt.title('train_loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.plot(history.history["loss"])

In [ ]:
predictions = model.predict(test_images)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability

In [ ]:
CM = confusion_matrix(test_labels, pred_labels)
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements 
print(accuracy(CM))

In [ ]:
ax = plt.axes()
sn.heatmap(CM, annot=True, 
           annot_kws={"size": 10}, 
           xticklabels=class_names, 
           yticklabels=class_names, ax = ax)
ax.set_title('Confusion matrix')
plt.show()

In [ ]:
from keras.models import load_model
# model.save("85%_model.h5")


In [ ]:
model = load_model('85%_model.h5')

In [ ]:
from keras.applications.xception import xception,decode_predictions
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
model = load_model('85%_model.h5')

In [ ]:
img_path='./0004362_person.jpg'
img = image.load_img(img_path, target_size=(64, 64))

In [ ]:
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
img_preprocessed = preprocess_input(img_batch)

In [ ]:
model = load_model('85%_model.h5',)

In [ ]:
prediction = model.predict(img_preprocessed)

In [ ]:
print(prediction)

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('85%_model.h5')

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt   
import numpy as np
import tensorflow as tf

In [ ]:
IMAGE_PATH='./0004362_person.jpg'

In [ ]:
img=tf.keras.preprocessing.image.load_img(IMAGE_PATH,
                                          target_size=(256,256))

In [ ]:
img=tf.keras.preprocessing.image.img_to_array(img)
plt.imshow(img/255.)

In [ ]:
predictions=model.predict(np.array([img]))

In [ ]:
# class_names = ['monkey','person']
# class_names[predictions]
print(predictions)